In [12]:
import os 
import glob
import gzip
import shutil
import torch
import rasterio
import pandas as pd 
import numpy as np
from tfrecord.torch.dataset import TFRecordDataset

In [13]:
CSV              = os.path.join( "..", "data", "wealth_index.csv" )
RECORDS_DIR      = os.path.join( "..", "data", "landsat_7", "" )
TIF_DIR          = os.path.join( "..", "data", "landsat_7","" )
BANDS            = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','TEMP1','NIGHTLIGHTS']
DESCRIPTOR       = {
                'system:index':"byte",
                'LAT':"byte", 
                "LON":"byte",
                'wealthpooled':"float",
                'BLUE':"float",
                'GREEN':"float",
                'RED':"float",
                'NIR':"float",
                'SWIR1':"float",
                'SWIR2':"float",
                'TEMP1':"float",
                'NIGHTLIGHTS':"float"
              }   

In [14]:
csv=pd.read_csv(CSV)

In [15]:
records = dict()
for year in csv.year.unique():
    records[year]=dict()
    sub_year = csv[ csv.year == year ]
    for country in sub_year.country.unique():
        sub_country = sub_year[ sub_year.country == country ].copy()
        pattern = RECORDS_DIR+"*"+str(country)+"_"+str(year)+"/*.tfrecord.gz"
        records[year][country] = glob.glob(pattern)
records[2011]['angola'][:5]

['../data/landsat_7/angola_2011/-12383679389953613.tfrecord.gz',
 '../data/landsat_7/angola_2011/-12394207954406738.tfrecord.gz',
 '../data/landsat_7/angola_2011/-12646769523620605.tfrecord.gz',
 '../data/landsat_7/angola_2011/-12614950180053711.tfrecord.gz',
 '../data/landsat_7/angola_2011/-12611531257629395.tfrecord.gz']

In [16]:
def decompress_tfrecord(tfrecord_archive):
    with gzip.open(tfrecord_archive, 'rb') as f_in:
        with open(tfrecord_archive[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return tfrecord_archive[:-3]

def tensor_to_string(data, variable):
    filename = (data[variable].numpy())[0]
    return "".join([chr(item) for item in filename])



def tfrecord_to_tif(data, filename, mins, maxs):
    arrays = []
    for i in range(len(BANDS)):
        new_arr = data[BANDS[i]][0].numpy().reshape((255,255))
        arrays.append(new_arr)
        mins[i] = min(mins[i], new_arr.min())
        maxs[i] = max(maxs[i], new_arr.max())

    arr = np.swapaxes(np.array(arrays), 0, 2 )
    tif_path = TIF_DIR + filename
    transform = rasterio.Affine(1, 0, 0, 0, 1, 0)
    tif = rasterio.open(tif_path, 'w', driver='GTiff',
                            height = arr.shape[0], width = arr.shape[1],
                            count=8, dtype=str(arr.dtype),
                            crs='epsg:3857',
                            transform=transform)
    for i in range(len(BANDS)):
        tif.write(arr[:,:,i],i+1)
    tif.close()

    return mins, maxs

In [44]:
mins=[1e3, 1e3, 1e3, 1e3, 1e3, 1e3, 1e3, 1e3]
maxs=[-1e3, -1e3, -1e3, -1e3, -1e3, -1e3, -1e3, -1e3]

for year in records:
    print(year)
    for country in records[year]:
        if records[year][country]==[]:
            continue
        for tfrecord in records[year][country]:
            tfrecord_path = decompress_tfrecord(tfrecord)
            dataset = TFRecordDataset(tfrecord_path, index_path=None, description=DESCRIPTOR)
            loader = torch.utils.data.DataLoader(dataset, batch_size=1)
            iterator = iter(loader)
            while (data := next(iterator, None)) is not None:
                filename = tensor_to_string(data, "LAT").replace(".","")+"_"+tensor_to_string(data, "LON").replace(".","")+".tif"
                print(filename)
                # mins, maxs = tfrecord_to_tif(data, filename, mins, maxs)

2011


/home/matthieu/anaconda3/envs/mpa_env/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


2015


KeyboardInterrupt: 

In [ ]:
print(mins, maxs)

[-0.0994, -0.0574, -0.0318, -0.0209, -0.0102, -0.0152, 0.0, -0.07087274] [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 316.7, 3104.1401]
